# Kansalliskirjaston leikkeiden tekstinlouhintaesimerkkejä (Digitalia-notebook)

Versiohistoria
* 0.3  7.2.2024 Leikkeiden lataaminen rajapinnan kautta, koodiin tarvittavia muutoksia (Erno Liukkonen)
* 0.2  30.9.2021 Aineiston lataaminen rajapinnan kautta, korjauksia koodiin ja käännöksiin (Erno Liukkonen) 
* 0.1  23.4.2019 Pieniä tekstikorjauksia
* 0.02 Digitalia-project, pieniä korjauksia
* 0.01 15.1.2019 (Mika Koistinen)


# Kiitokset



<table><tr><td>
<img src="https://blogs.helsinki.fi/digitalia/files/2018/10/sosiaali_fi_90p.jpg" style="height:100px;width:100%">
    </td><td>
<img src="https://blogs.helsinki.fi/digitalia/files/2018/10/fi_EU_rgb_90p.jpg" style="height:100px;width:100%">
    </td><td>
<img src="https://blogs.helsinki.fi/digitalia/files/2015/10/digitalia_pien_512.png" style="height:70px;width:100%">
    </td></td></tr></table>

# Leikkeiden lataaminen rajapinnan kautta

Leikkeiden lataaminen tapahtuu rajapintaa hyödyntäen. Suorita leikkeiden haku https://digi.kansalliskirjasto.fi/clippings osoitteessa ja kopioi hakutulokset sisältävän sivun osoite alla olevaan searchResultsUrl muuttujaan (oletuksena nyt https://digi.kansalliskirjasto.fi/clippings?formats=NEWSPAPER&startDate=1890-08-01&endDate=1890-09-01&title=1458-2457&resultMode=THUMB).

clippingsSearchQuery funktio suorittaa haettujen leikkeiden tietojen lataamisen rajapinnan kautta. Funktiolle annetaan parametrinä hakutulokset sisältävän sivun osoite.

In [ ]:
# -*- coding: utf-8 -*-

import urllib
from urllib.error import URLError, HTTPError
from socket import timeout
import simplejson as json

searchResultsUrl = "https://digi.kansalliskirjasto.fi/clippings?formats=NEWSPAPER&startDate=1890-08-01&endDate=1890-09-01&title=1458-2457&resultMode=THUMB"

request_headers = {
"User-Agent": "Notebook dam-rajapinta",
"Referer": "Notebook dam-rajapinta",
"Connection": "keep-alive" 
}

def clippingsSearchQuery(digiResultsUrl):

    parameters = digiResultsUrl[digiResultsUrl.index('?'):]

    currentRows = []
    result = ""
    noMoreResults = False
    isFirstSearch = True

    digiClippingsSearchURL = 'https://digi.kansalliskirjasto.fi/api/dam/clippings-search/' + parameters

    while noMoreResults == False:
    
      req = urllib.request.Request(digiClippingsSearchURL, headers=request_headers)
      try:
        response = urllib.request.urlopen(req, timeout=30)
        responseResult = response.read()
        result = json.loads(responseResult)

        if len(result["rows"]) != 0:
          currentRows = currentRows + result["rows"]
        else:
          noMoreResults = True

        if isFirstSearch == True:
          digiClippingsSearchURL = 'https://digi.kansalliskirjasto.fi/api/dam/clippings-search/' + result["scrollId"]
          isFirstSearch = False

      except HTTPError as e:
        content = e.read()
      except ConnectionError as e:
        print("Ei yhteyttä palvelimeen!")
      except URLError as e:
        print("Ei yhteyttä palvelimeen!")
      except TimeoutError as e:
        print("Ei yhteyttä palvelimeen!")
      except timeout:
        print("Ei yhteyttä palvelimeen!")

    result["rows"] = currentRows

    return result

Seuraavaksi suoritetaan varsinainen leikkeiden teksti- ja kuvatiedostojen lataus urlretrieve funktion avulla. Tiedostojen latauksissa tarvittavien tietojen muodostamisessa käytetään clippingsSearchQuery funktion kautta saatuja tietoja. Tiedostot ladataan oletuksena downloads-hakemistoon, joka sijaitsee samassa hakemistossa mistä notebook käynnistettiin.

In [ ]:
import sys
import os
import urllib
import simplejson as json

def urlretrieve(url, localfile):

    try:
        req = urllib.request.Request(url, headers=request_headers)
        response = urllib.request.urlopen(req, timeout=30)
        responseResult = response.read()

        if localfile.endswith(".txt"):
            tContent = json.loads(responseResult)
            with open(localfile, "w", encoding="utf-8") as fl:
              fl.write(' '.join(tContent))
        else:
            with open(localfile, "wb") as fl:
              fl.write(responseResult)

    #handle errors
    except Exception as e:
      print("Ei yhteyttä palvelimeen!") + " " + str(e)
      return 0

    if not os.path.exists(localfile):
      return 0

    return 1

notebookPath =  os.path.dirname(os.path.realpath("__file__"))

result = clippingsSearchQuery(searchResultsUrl)

rows = result["rows"]

isFirstResultRow = True
firstResultTxt = ""
firstResultPath = ""
 
resultCounter = 1

for row in rows:
    data = row["data"] 
    bindingTitle = data["bindingTitle"]
    bindingId = str(data["bindingId"])
    articleId = str(data["articleId"])
    
    ocrTextUrl = row["ocrTextUrl"]
    currentImageUrls = []
    currentArticleRegionIds = row["articleRegionIds"].split(',')
    currentPageNumbers = row["articleRegionPageNumbers"].split(',')

    if "imageUrlTemplate" in row and row["imageUrlTemplate"] != None:
          if "articleRegionIds" in row and row["articleRegionIds"] != None:
            for regionId in row["articleRegionIds"].split(','):
                currentImageUrls.append(row["imageUrlTemplate"].replace("{{regionId}}", regionId))
    
    txtPath = notebookPath + "/downloads/" + bindingTitle + "_" + bindingId + "_" + articleId + "/txt/"
    imgPath = notebookPath + "/downloads/" + bindingTitle + "_" + bindingId + "_" + articleId +"/jpg/"

    if not os.path.exists(txtPath):
        os.makedirs(txtPath)
    
    if not os.path.exists(imgPath):
        os.makedirs(imgPath)
        
    downloadPathTxt = txtPath + bindingTitle + "_" + bindingId + "_" + articleId + "_page_" + currentPageNumbers[0] + ".txt"
    
    print("Ladataan tulosta: " + str(resultCounter) + "/" + str(len(rows)))
    
    urlretrieve(ocrTextUrl, downloadPathTxt)
    
    for index, currentImageUrl in enumerate(currentImageUrls):
        downloadPathJpg = imgPath + bindingTitle + "_" + bindingId + "_" + articleId + "_" + currentArticleRegionIds[index] + "_page_" + currentPageNumbers[index] + ".jpg"
        urlretrieve(currentImageUrl, downloadPathJpg)
    
    resultCounter += 1
    
    if isFirstResultRow:
        firstResultTxt = downloadPathTxt
        firstResultPath = txtPath
        isFirstResultRow = False

# Luetaan yksi ladatuista leikkeiden tekstitiedostoista

In [ ]:
import os

global firstResultPath

#Ensimmäisen ladatun leikkeen tekstitiedoston polku, joka rajapinnan kautta ladattiin
path = firstResultPath 

print(path)


In [ ]:
#tällä voi katsoa mitä tiedostoja kansiossa:
sorted(os.listdir(path)) 

In [ ]:
from bs4 import BeautifulSoup as bs

global firstResultTxt

#luetaan ensimmäisen ladatun leikkeen teksti
filename = firstResultTxt
with open(firstResultTxt, 'r', encoding="utf-8") as f:
        content = f.read()
    

In [ ]:
#luodaan sanatokenit
#NLTK
#pip install nltk
import nltk 
tokens=nltk.word_tokenize(content)
tokens[0:10]

# Word Frequencies /sanojen esiintymistiheydet

In [ ]:
#removing stopwords
from nltk.corpus import stopwords
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#sulkusanat
stop_words=set(stopwords.words('finnish'))
#print(stopwords[0:10],len(stopwords))
#filtered_sentence = [w for w in content if not w in stop_words]

#sulkusanat poistetaan
filtered_content=[]
for r in tokens:
    #print(r)
    if not r in stop_words:
        filtered_content.append(r)

#sulkusanat alle 1 merkin sanat ja yli 20 merkin sanat poistetaan
filtered_content2=[]
for r in tokens:
    #print(r)
    if not r in stop_words and len(r) <20 and len(r)>1: # and
        filtered_content2.append(r)
        

In [ ]:
#tulostetaan osa eritavoin esikäsitellyistä tokeneista
freqz=nltk.FreqDist(tokens)
freqz2=nltk.FreqDist(filtered_content)
freqz3=nltk.FreqDist(filtered_content2)

print("koko teksti")
print(freqz.most_common()[0:15])
print("stopwords poistettu")
print(freqz2.most_common()[0:15])



# WordClouds / sanapilvet 

In [ ]:
#löytyy täältä esimerkkiä 
#https://github.com/amueller/word_cloud
#!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


tokens_str = ' '.join(tokens)
fig = plt.figure(figsize=(24, 14)) 
wordcloud=WordCloud().generate(tokens_str)
#wordcloud = WordCloud(max_font_size=50,min_font_size=5).generate(content_str)
plt.subplot(221)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("1 document (all tokens)")
plt.show()


fig = plt.figure(figsize=(24, 14)) 
content_str2 = ' '.join(filtered_content)
wordcloud2=WordCloud().generate(content_str2)
#wordcloud2 = WordCloud(max_font_size=40,min_font_size=5).generate(content_str2)
plt.subplot(222)
plt.imshow(wordcloud2, interpolation="bilinear")
plt.title("1 document (removed stopwords)")


plt.axis("off")
plt.show()


print("sanojen määrä:",len(tokens),len(filtered_content))#,len(content_str3))
#content_str2




# Useampi tiedosto kansiosta


In [ ]:
# loading all text files from the folders / ladataan kaikki tekstitiedostot kansiosta

content = []
docs_complete = []

for paths, dirs, files in os.walk(notebookPath + "/downloads/"):
    for file in files:
        if file.endswith('.txt'):
            with open(os.path.join(paths,file), 'r', encoding="utf-8") as f:
                if len(content) != 0:
                    content.append(' ')
                currentContent = f.read().lower().split(' ')
                content += currentContent
                docs_complete.append(currentContent)
            
print("sanojen määrä kaikki leikkeet kansiossa:",len(content))


In [ ]:
#poistetaan alle 2 merkin pituiset sanat
content2=[stri2 for stri2 in content if len(stri2)>1]
len(content2)

In [ ]:
#poistetaan ohitettavat sanat (stopwords)
filtered_content3=[]
for r in content2:
    #print(r)
    if not r in stop_words:
        filtered_content3.append(r)
len(filtered_content3)

In [ ]:
freqz3=nltk.FreqDist(content2)
freqz3.most_common()[0:15]

freqz4=nltk.FreqDist(filtered_content3)

print(freqz3.most_common()[0:10])
print(freqz4.most_common()[0:10])



# WordClouds / sanapilvet (kaikki hakemistossa olevat sivut)

In [ ]:
content_str2 = ' '.join(content2)



fig = plt.figure(figsize=(16, 10))
plt.subplot(223)
wordcloud3 = WordCloud(max_font_size=50,min_font_size=5).generate(content_str2)

plt.imshow(wordcloud3, interpolation="bilinear")
plt.title("all "+str(len(docs_complete))+" documents, and "+str(len(content2))+" words, from folder")

plt.axis("off")


content_str3 = ' '.join(filtered_content3)

plt.subplot(224)
wordcloud4 = WordCloud(max_font_size=50,min_font_size=5).generate(content_str3)
plt.imshow(wordcloud4, interpolation="bilinear")

plt.title("all "+str(len(docs_complete))+" documents, and "+str(len(filtered_content3))+" words, from folder (removed stopwords)")

plt.axis("off")
plt.show()

# Topic Modeling / Aihemallinnus

In [ ]:
#based on 
#https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

len(docs_complete),len(content2),len(filtered_content3)



In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('finnish'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    doc2=[word for word in doc if len(word)>3]       
    stop_free = " ".join([i for i in doc2 if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

    
doc_clean = [clean(doc).split() for doc in docs_complete]        

In [ ]:
#doc_clean=docs2#filtered_content2#map(unicode,filtered_content2)
# Importing Gensim
import gensim
from gensim import corpora
dictionary = corpora.Dictionary(doc_clean )
#dictionary
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]



In [ ]:
# Creating the object for LDA model using gensim library

Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=100, id2word = dictionary, passes=50)

In [ ]:
#Results
print(ldamodel.print_topics(num_topics=100, num_words=3))


In [ ]:
print(ldamodel.print_topics(num_topics=10, num_words=5))

In [ ]:
len(docs_complete),len(doc_clean)

TOPIC MODELING2 https://de.dariah.eu/tatom/topic_model_python.html

In [ ]:
#FILES
FILES2=[]
        
for paths, dirs, files in os.walk(notebookPath + "/downloads/"):
    for file in files:
        if file.endswith('.txt'):
            FILES2.append(os.path.join(paths,file))
            print(FILES2[-1])


In [ ]:
############

import numpy as np
#FILES[0:4]
stop = list(set(stopwords.words('finnish')))

import sklearn.feature_extraction.text as text
vectorizer = text.CountVectorizer(input='filename', stop_words=stop, min_df=1)
dtm = vectorizer.fit_transform(FILES2).toarray()
vocab = np.array(vectorizer.get_feature_names())


In [ ]:
dtm.shape,len(vocab)

In [ ]:
from sklearn import decomposition
num_topics = 10
num_top_words = 10
clf = decomposition.NMF(n_components=num_topics, random_state=1)



In [ ]:
doctopic = clf.fit_transform(dtm)

In [ ]:
topic_words = []
for topic in clf.components_:
   word_idx = np.argsort(topic)[::-1][0:num_top_words]
   topic_words.append([vocab[i] for i in word_idx])

In [ ]:
doctopic = doctopic / np.sum(doctopic, axis=1, keepdims=True)

In [ ]:
novel_names = []

# turn this into an array so we can use NumPy functions
#novel_names = np.asarray(novel_names)
novel_names=np.asarray(FILES2)
doctopic_orig = doctopic.copy()

# use method described in preprocessing section
num_groups = len(set(novel_names))
doctopic_grouped = np.zeros((num_groups, num_topics))

for i, name in enumerate(sorted(set(novel_names))):
        doctopic_grouped[i, :] = np.mean(doctopic[novel_names == name, :], axis=0)

doctopic = doctopic_grouped

In [ ]:
novels = sorted(set(novel_names))

print("Top NMF topics in...")


for i in range(len(doctopic)):
        top_topics = np.argsort(doctopic[i,:])[::-1][0:3]
        top_topics_str = ' '.join(str(t) for t in top_topics)
        print("{}: {}".format(novels[i], top_topics_str))
    


In [ ]:
for t in range(len(topic_words)):
   print("Topic {}: {}".format(t, ' '.join(topic_words[t][:15])))